In [71]:
import pandas as pd
import numpy as np
import xgboost as xgb
import optuna

In [39]:
test_data = pd.read_csv('/Users/baconbaker/Documents/Studium/ANM/anm-project/data/test_data/esb.csv')
train_data = pd.read_csv('/Users/baconbaker/Documents/Studium/ANM/anm-project/data/train_data/esb.csv')

print(test_data.head())
print(train_data.head())

  service_name     start_time  avg_time  num  succee_num  succee_rate
0      osb_001  1606881600000    0.6214  361         361       1.0000
1      osb_001  1606881660000    0.5861  343         343       1.0000
2      osb_001  1606881720000    5.0125  126         122       0.9683
3      osb_001  1606881780000    9.3487   70          70       1.0000
4      osb_001  1606881840000    1.4756  375         375       1.0000
  serviceName      startTime  avg_time  num  succee_num  succee_rate
0     osb_001  1588521600000    0.5691  347         347          1.0
1     osb_001  1588521660000    0.5813  354         354          1.0
2     osb_001  1588521720000    0.5397  363         363          1.0
3     osb_001  1588521780000    0.6190  387         387          1.0
4     osb_001  1588521840000    0.4909  387         387          1.0


In [40]:
#Reformatting train data
train_data['start_time'] = train_data['startTime']

In [41]:
def get_onehot(df):
    df['start_time'] = pd.to_datetime(df['start_time'], unit='ms', origin='unix')
    df['start_hour'] = df["start_time"].dt.strftime("%H")
    df['start_minute'] = df["start_time"].dt.strftime("%M")
    return df

def split(df):
    df_x = df[['start_hour', 'start_minute']].astype('int32')
    df_y = df[['num', 'avg_time']]
    return df_x, df_y

In [42]:
train_data = get_onehot(train_data)
train_x, train_y = split(train_data)

test_data = get_onehot(test_data)
test_x, test_y = split(test_data)

In [69]:
def objective(trial):
    param = {
        'max_depth': trial.suggest_int('depth', 2, 20),
        'eta': trial.suggest_float('eta', .8, 2),
        'objective': 'reg:squaredlogerror'}

    param['eval_metric'] = 'rmsle'
    evallist = [(dtest, 'eval'), (dtrain, 'train')]

    num_round = tri
    model = xgb.train(param, dtrain, num_round, evallist, evals_result=progress)
    return progress['eval']['rmsle'][-1]
    

In [72]:
study = optuna.create_study()
study.optimize(objective, n_trials=100)

param = study.best_params  # E.g. {'x': 2.002108042}
print(param)

[I 2020-12-03 18:20:49,171] A new study created in memory with name: no-name-9c63eaab-30d9-4aa0-9dc5-3bde5ae67581


[0]	eval-rmsle:0.27515	train-rmsle:0.04924
[1]	eval-rmsle:0.29991	train-rmsle:0.04751
[2]	eval-rmsle:0.27825	train-rmsle:0.03661
[3]	eval-rmsle:0.29289	train-rmsle:0.03348
[4]	eval-rmsle:0.28245	train-rmsle:0.02665
[5]	eval-rmsle:0.28799	train-rmsle:0.02421
[6]	eval-rmsle:0.28420	train-rmsle:0.02096
[7]	eval-rmsle:0.28729	train-rmsle:0.01921
[8]	eval-rmsle:0.28491	train-rmsle:0.01715
[9]	eval-rmsle:0.28619	train-rmsle:0.01551
[10]	eval-rmsle:0.28555	train-rmsle:0.01448
[11]	eval-rmsle:0.28621	train-rmsle:0.01376
[12]	eval-rmsle:0.28595	train-rmsle:0.01263
[13]	eval-rmsle:0.28590	train-rmsle:0.01169
[14]	eval-rmsle:0.28561	train-rmsle:0.01117
[15]	eval-rmsle:0.28591	train-rmsle:0.01071
[16]	eval-rmsle:0.28586	train-rmsle:0.01026
[17]	eval-rmsle:0.28581	train-rmsle:0.00973
[18]	eval-rmsle:0.28581	train-rmsle:0.00932
[19]	eval-rmsle:0.28574	train-rmsle:0.00903
[20]	eval-rmsle:0.28571	train-rmsle:0.00862
[21]	eval-rmsle:0.28586	train-rmsle:0.00827
[22]	eval-rmsle:0.28575	train-rmsle:0.0080

[I 2020-12-03 18:20:49,397] Trial 0 finished with value: 0.285876 and parameters: {'depth': 9, 'eta': 1.833227450438662}. Best is trial 0 with value: 0.285876.


[0]	eval-rmsle:0.28112	train-rmsle:0.02874
[1]	eval-rmsle:0.28537	train-rmsle:0.02123
[2]	eval-rmsle:0.28521	train-rmsle:0.01706
[3]	eval-rmsle:0.28536	train-rmsle:0.01591
[4]	eval-rmsle:0.28539	train-rmsle:0.01465
[5]	eval-rmsle:0.28543	train-rmsle:0.01361
[6]	eval-rmsle:0.28583	train-rmsle:0.01235
[7]	eval-rmsle:0.28592	train-rmsle:0.01160
[8]	eval-rmsle:0.28574	train-rmsle:0.01090
[9]	eval-rmsle:0.28556	train-rmsle:0.01059
[10]	eval-rmsle:0.28560	train-rmsle:0.00981
[11]	eval-rmsle:0.28552	train-rmsle:0.00947
[12]	eval-rmsle:0.28555	train-rmsle:0.00898
[13]	eval-rmsle:0.28562	train-rmsle:0.00867
[14]	eval-rmsle:0.28564	train-rmsle:0.00834
[15]	eval-rmsle:0.28566	train-rmsle:0.00794
[16]	eval-rmsle:0.28570	train-rmsle:0.00768
[17]	eval-rmsle:0.28583	train-rmsle:0.00733
[18]	eval-rmsle:0.28572	train-rmsle:0.00709
[19]	eval-rmsle:0.28570	train-rmsle:0.00686
[20]	eval-rmsle:0.28568	train-rmsle:0.00669
[21]	eval-rmsle:0.28571	train-rmsle:0.00643
[22]	eval-rmsle:0.28575	train-rmsle:0.0062

[I 2020-12-03 18:20:49,546] Trial 1 finished with value: 0.285812 and parameters: {'depth': 12, 'eta': 1.3324104654306683}. Best is trial 1 with value: 0.285812.


...

[0]	eval-rmsle:0.28503	train-rmsle:0.02559
[1]	eval-rmsle:0.28523	train-rmsle:0.02532
[2]	eval-rmsle:0.28497	train-rmsle:0.02497
[3]	eval-rmsle:0.28484	train-rmsle:0.02471
[4]	eval-rmsle:0.28490	train-rmsle:0.02456
[5]	eval-rmsle:0.28502	train-rmsle:0.02441
[6]	eval-rmsle:0.28483	train-rmsle:0.02432
[7]	eval-rmsle:0.28485	train-rmsle:0.02414
[8]	eval-rmsle:0.28471	train-rmsle:0.02405
[9]	eval-rmsle:0.28474	train-rmsle:0.02397
[10]	eval-rmsle:0.28475	train-rmsle:0.02392
[11]	eval-rmsle:0.28493	train-rmsle:0.02384
[12]	eval-rmsle:0.28489	train-rmsle:0.02376
[13]	eval-rmsle:0.28504	train-rmsle:0.02369
[14]	eval-rmsle:0.28476	train-rmsle:0.02354
[15]	eval-rmsle:0.28490	train-rmsle:0.02345
[16]	eval-rmsle:0.28504	train-rmsle:0.02337
[17]	eval-rmsle:0.28502	train-rmsle:0.02333
[18]	eval-rmsle:0.28532	train-rmsle:0.02327
[19]	eval-rmsle:0.28518	train-rmsle:0.02324
[20]	eval-rmsle:0.28520	train-rmsle:0.02318
[21]	eval-rmsle:0.28516	train-rmsle:0.02313
[22]	eval-rmsle:0.28498	train-rmsle:0.0230

[I 2020-12-03 18:21:00,471] Trial 98 finished with value: 0.285034 and parameters: {'depth': 2, 'eta': 1.0926045013730799}. Best is trial 2 with value: 0.284041.


[0]	eval-rmsle:0.27859	train-rmsle:0.03495
[1]	eval-rmsle:0.29040	train-rmsle:0.02862
[2]	eval-rmsle:0.28365	train-rmsle:0.02530
[3]	eval-rmsle:0.28591	train-rmsle:0.02439
[4]	eval-rmsle:0.28483	train-rmsle:0.02399
[5]	eval-rmsle:0.28536	train-rmsle:0.02377
[6]	eval-rmsle:0.28445	train-rmsle:0.02347
[7]	eval-rmsle:0.28488	train-rmsle:0.02329
[8]	eval-rmsle:0.28498	train-rmsle:0.02309
[9]	eval-rmsle:0.28489	train-rmsle:0.02297
[10]	eval-rmsle:0.28488	train-rmsle:0.02280
[11]	eval-rmsle:0.28465	train-rmsle:0.02261
[12]	eval-rmsle:0.28475	train-rmsle:0.02233
[13]	eval-rmsle:0.28460	train-rmsle:0.02219
[14]	eval-rmsle:0.28484	train-rmsle:0.02211
[15]	eval-rmsle:0.28498	train-rmsle:0.02204
[16]	eval-rmsle:0.28475	train-rmsle:0.02198
[17]	eval-rmsle:0.28462	train-rmsle:0.02185
[18]	eval-rmsle:0.28461	train-rmsle:0.02174
[19]	eval-rmsle:0.28440	train-rmsle:0.02168
[20]	eval-rmsle:0.28453	train-rmsle:0.02159
[21]	eval-rmsle:0.28446	train-rmsle:0.02151
[22]	eval-rmsle:0.28443	train-rmsle:0.0214

[I 2020-12-03 18:21:00,572] Trial 99 finished with value: 0.284762 and parameters: {'depth': 3, 'eta': 1.5158624709537087}. Best is trial 2 with value: 0.284041.


{'depth': 2, 'eta': 1.6486872914846953}


In [ ]:
dtrain = xgb.DMatrix(train_x, label=train_y[['avg_time']])
dtest = xgb.DMatrix(test_x, label=test_y[['avg_time']])

param = {
    'max_depth': 3,
    'eta': 1.6487,
    'objective': 'reg:squaredlogerror'}

param['eval_metric'] = 'rmsle'
evallist = [(dtest, 'eval'), (dtrain, 'train')]
progress = {}

num_round = 60
model = xgb.train(param, dtrain, num_round, evallist, evals_result=progress)
print(progress['eval']['rmsle'][-1])